# Put the pieces together. 

In [62]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from providers import PROVIDERS
import check_available as ck
from time import sleep
import send_text as txt
import datetime

### Connect

In [63]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
URL = 'https://disneyworld.disney.go.com/availability-calendar/?segments=tickets,resort,passholder&defaultSegment=tickets'
browser.get(URL)
driver = browser
sleep(2)

### Pick Vacation Date Range 

In [64]:
firstday = input('What is the the first day of Disney?  mm/dd/yyyy\n')     # \n ---> newline  ---> It causes a line break
first = datetime.datetime.strptime(firstday, '%m/%d/%Y')

What is the the first day of Disney?  mm/dd/yyyy
07/13/2022


In [65]:
lastday = input('What is the the first day of Disney?  mm/dd/yyyy\n')     # \n ---> newline  ---> It causes a line break
last = datetime.datetime.strptime(lastday, '%m/%d/%Y')

What is the the first day of Disney?  mm/dd/yyyy
07/16/2022


### Extract the dates 

In [66]:
def diz_range(start,stop):
#     global diz_dates # If you want to use this outside of functions     
    diz_dates = []
    diff = (stop-start).days
    for i in range(diff+1):
        day = start + datetime.timedelta(days=i)
        diz_dates.append(day)
    return diz_dates

In [67]:
diz_dates = diz_range(first,last)
diz_dates

[datetime.datetime(2022, 7, 13, 0, 0),
 datetime.datetime(2022, 7, 14, 0, 0),
 datetime.datetime(2022, 7, 15, 0, 0),
 datetime.datetime(2022, 7, 16, 0, 0)]

### Add date full names to dates 

In [68]:
def Park_Picker(diz_dates):
    global park_dates
    park_dates ={}    
    num = len(diz_dates)
    while num > 0:
        for d in diz_dates:
            num=-1
            p_date = d.strftime("%A, %B %d, %Y")
            park = input('What park are you going to on ' + str(p_date)+'?  MK , EP , AK , HS \n') 
            if park.upper() == 'MK':
                park_dates[p_date] = 'Magic Kingdom'
            elif park.upper() == 'EP':
                park_dates[p_date] = 'EPCOT'
            elif park.upper() == 'AK':
                park_dates[p_date] = 'Animal Kingdom'
            elif park.upper() == 'HS':
                park_dates[p_date] = 'Hollywood Studios'        
            next
    return park_dates

In [69]:
Park_Picker(diz_dates)

What park are you going to on Wednesday, July 13, 2022?  MK , EP , AK , HS 
AK
What park are you going to on Thursday, July 14, 2022?  MK , EP , AK , HS 
MK
What park are you going to on Friday, July 15, 2022?  MK , EP , AK , HS 
HS
What park are you going to on Saturday, July 16, 2022?  MK , EP , AK , HS 
EP


{'Wednesday, July 13, 2022': 'Animal Kingdom',
 'Thursday, July 14, 2022': 'Magic Kingdom',
 'Friday, July 15, 2022': 'Hollywood Studios',
 'Saturday, July 16, 2022': 'EPCOT'}

### Query all dates on the calendar

In [70]:
# def available(driver,day,park):
alert = None
park_list = []

CheckAll = """ return document
.querySelector("#container > app-availability-calendar > awakening-calendar").shadowRoot
.querySelectorAll("#calendarContainer > wdat-calendar > wdat-date")"""

all_dates= driver.execute_script(CheckAll)

### create dictionary of all dates -> click_elements based on vacation date

In [71]:
# Need to check in this value is IN the date list 
pdlist = park_dates.keys()
# this creates the list of elements to click per day
date_ele = {}
for date in all_dates:
    new = date.get_attribute("aria-label").replace('selected ','')
    if new in pdlist:
        # if the value is in the list then save as key date and element
        date_ele[new] = date

### click dates in date_ele click and check availability  returns park_avail

In [95]:
# takes in the date list and selenium elements
# clicks on each day returns days that have reservations and days that done
# returns globals park_avail and not_avail
def find_park_avail(pdlist,date_ele):
    global park_avail
    global not_avail
    park_avail = {} # for each date click 
    for d in pdlist:
        availpark_list = []
        not_list = []
        try:
            date_ele[d].click()
            # if does not save to not_avail, else save to park_avail
            for i in range(1,5):     
                sub = 'does not have'
                query= """return document
                .querySelector("#container > app-availability-calendar > awakening-availability-information").shadowRoot
                .querySelector("#parkAvailabilityContainer > div:nth-child("""+str(i)+""") > dprd-icon")"""
                status = driver.execute_script(query).get_attribute("aria-label")
                if sub not in status:
                    availpark_list.append(status)
                else:
                    not_list.append(status)
            # save availibility and date dictionary
            park_avail[d] = availpark_list
            not_avail[d] = not_list
        except: 
            None 

In [96]:
find_park_avail(pdlist,date_ele)

### Find days that are not available that you wanted

In [98]:
def alert_needed(park_dates,not_avail):
    alert = {}
    for d in park_dates.keys():
        park_pick = park_dates[d]
        for n in not_avail[d]:
            if park_pick in n:
                alert[d] = park_pick
    return alert

In [103]:
Create_alert = alert_needed(park_dates,not_avail)

### Need to create the actual alert

In [101]:
# So unlike the other one this one needs state. So when it runs every 5 min it needs to pull this alert up. 

In [104]:
import json 

In [106]:
# save to file 
alert_file = json.dumps(alert_needed(park_dates,not_avail))
avail_file = json.dumps(park_avail)
not_avail_file = json.dumps(not_avail, ensure_ascii=False).encode('utf8')

In [117]:
not_avail_file = json.dumps(not_avail, ensure_ascii=False).encode('utf8')
print(not_avail_file)

b'{"Wednesday, July 13, 2022": ["Magic Kingdom does not have reservations available.", "Disney\xe2\x80\x99s Animal Kingdom does not have reservations available.", "Disney\xe2\x80\x99s Hollywood Studios does not have reservations available."], "Thursday, July 14, 2022": ["Magic Kingdom does not have reservations available.", "Disney\xe2\x80\x99s Animal Kingdom does not have reservations available.", "Disney\xe2\x80\x99s Hollywood Studios does not have reservations available."], "Friday, July 15, 2022": ["Magic Kingdom does not have reservations available.", "Disney\xe2\x80\x99s Hollywood Studios does not have reservations available."], "Saturday, July 16, 2022": []}'


# This should be the new script that runs in cron / airflow